In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import xarray as xr
import scipy.stats

### Load and sort data

In [2]:
### Identify sample files
ukl_in = Path('/projects/wakedynamics/orybchuk/ldm-3d/logs/2022-12-09T09-20-20_test_dm_geo_uncond_kl1/images/test/postprocessed')
ukl_files = list(Path(ukl_in).glob('*.npy'))
ukl_files.sort()

In [3]:
### Prepare four large sample arrays
# nubatch = min(len(ucont_files), len(ucontp_files), len(ukl_files), len(uklp_files))
nubatch = len(ukl_files)
tmp_samp = np.load(ukl_files[0])
ucond_shape = [nubatch]+list(tmp_samp.shape)

tmp_kl = np.zeros(ucond_shape)
for i in range(nubatch):
    tmp_kl[i,:,:,:,:] = np.load(ukl_files[i]).copy()

In [4]:
### Convert to Dataset
## Prepare metadata
xdim = np.arange(0, 1920, 15)
ydim = np.arange(0, 1920, 15)
zdim = np.arange(0, 480, 15)  # Chop in half to avoid looking above the capping inversion
coords = dict(x=xdim, y=ydim, z=zdim)

## Create Dataset
ds_u = xr.Dataset(coords=coords)

ds_u['u_kl'] = (('sample', 'x', 'y', 'z'), tmp_kl[:,0,:,:,:32])
ds_u['v_kl'] = (('sample', 'x', 'y', 'z'), tmp_kl[:,1,:,:,:32])
ds_u['w_kl'] = (('sample', 'x', 'y', 'z'), tmp_kl[:,2,:,:,:32])

## Un-normalize data
def unnorm(xp, xmin, xmax):
    '''
    Take data from [-1,1] back to the original values
    '''
    return (xp + 1)*0.5*(xmax-xmin)+xmin

umin, umax, vmin, vmax, wmin, wmax = 2.86975098, 12.5567627, -0.9810791, 4.91235352, -1.98095703, 2.5579834
ds_u['u_kl'] = unnorm(ds_u['u_kl'], umin, umax)
ds_u['v_kl'] = unnorm(ds_u['v_kl'], vmin, vmax)
ds_u['w_kl'] = unnorm(ds_u['w_kl'], wmin, wmax)

### Velocity profiles

In [5]:
### Calculate stats
for vel in ['u', 'v', 'w']:
    ds_u[vel+'_kl'].mean(('sample', 'x', 'y')).to_netcdf(vel+'_ucond_kl_mean.nc')
    ds_u[vel+'_kl'].std(('sample', 'x', 'y')).to_netcdf(vel+'_ucond_kl_std.nc')

### Flux profiles

In [ ]:
### Calculate profiles of kinematic flux
## Calculate fluctuations
ds_u["up_kl"] = ds_u['u_kl'] - ds_u['u_kl'].mean(('x', 'y'))
ds_u["vp_kl"] = ds_u['v_kl'] - ds_u['v_kl'].mean(('x', 'y'))
ds_u["wp_kl"] = ds_u['w_kl'] - ds_u['w_kl'].mean(('x', 'y'))

## Calculate fluxes
ds_u["upvp_kl"] = (ds_u['up_kl'] * ds_u['vp_kl']).mean(('x', 'y'))
ds_u["upwp_kl"] = (ds_u['up_kl'] * ds_u['wp_kl']).mean(('x', 'y'))
ds_u["vpwp_kl"] = (ds_u['vp_kl'] * ds_u['wp_kl']).mean(('x', 'y'))

# ## Save
# ds_u["upvp_kl"].mean('sample').to_netcdf("upvp_ucond_kl.nc")
# ds_u["upwp_kl"].mean('sample').to_netcdf("upwp_ucond_kl.nc")
# ds_u["vpwp_kl"].mean('sample').to_netcdf("vpwp_ucond_kl.nc")

### Velocity PDFs

In [ ]:
### Calculate histograms and PDFs
## Bins
umin = 6
umax = 12
ubins = np.linspace(umin, umax, 500)

vmin = -1.5
vmax = 3.5
vbins = np.linspace(vmin, vmax, 500)

wmin = -0.75
wmax = 0.75
wbins = np.linspace(wmin, wmax, 300)

## Scipy histograms
uhist_np_kl = np.histogram(ds_u['u_kl'].values.flatten(), bins=ubins)
uhist_kl = scipy.stats.rv_histogram(uhist_np_kl)

vhist_np_kl = np.histogram(ds_u['v_kl'].values.flatten(), bins=vbins)
vhist_kl = scipy.stats.rv_histogram(vhist_np_kl)

whist_np_kl = np.histogram(ds_u['w_kl'].values.flatten(), bins=wbins)
whist_kl = scipy.stats.rv_histogram(whist_np_kl)

## Save
# np.save('u_ucond_kl_hist.npy', uhist_kl.pdf(ubins))

# np.save('v_ucond_kl_hist.npy', vhist_kl.pdf(vbins))

# np.save('w_ucond_kl_hist.npy', whist_kl.pdf(wbins))

In [ ]:
len(whist_np_kl[0])

### Spectra

In [ ]:
### Calculate 1D spectra, sample by sample
khub = 6
tmp1 = ds_u['up_kl'].isel(sample=0, z=khub)
tmp2 = np.fft.rfftn(tmp1.values, axes=(0,))
all_spectra_kl = np.zeros((len(ds_u['sample']), len(tmp2)))
    
for i in range(len(ds_u['sample'])):
    up_xy_test = ds_u['up_kl'].isel(sample=i, z=khub)
    up_k_test = np.fft.rfftn(up_xy_test.values, axes=(0,))
    up_k_bar_test = np.mean(np.abs(up_k_test)**2, axis=1)
    all_spectra_kl[i,:] = up_k_bar_test.copy()

k_u = np.fft.rfftfreq(np.shape(up_xy_test)[0])

# np.save('spectra_ucond_kl.npy', all_spectra_kl.mean(axis=0))
# np.save('spectra_k_u.npy', k_u)

### Continuity

In [ ]:
### Calculate pdf's of gradients + divergence
## Parameters
dx, dy, dz = 15, 15, 15
facx, facy, facz = 1/(4*dx), 1/(4*dy), 1/(4*dz)
dmin = -0.02
dmax = 0.02
dbins = np.linspace(dmin, dmax, 500)

## Set up history arrays

all_dudx_hist_kl = np.zeros((len(ds_u['sample']), len(dbins)))
all_dvdy_hist_kl = np.zeros((len(ds_u['sample']), len(dbins)))
all_dwdz_hist_kl = np.zeros((len(ds_u['sample']), len(dbins)))
all_div_hist_kl = np.zeros((len(ds_u['sample']), len(dbins)))

## Calculate pdf's
for i in range(len(ds_u['sample'])):
    vel = ds_u.isel(sample=i)

    
    dudx_kl = facx*(-vel['u_kl'].roll(x=-1,y=-1,z=-1).values + vel['u_kl'].roll(x=0,y=-1,z=-1).values \
                    -vel['u_kl'].roll(x=-1,y=0 ,z=-1).values + vel['u_kl'].roll(x=0,y=0 ,z=-1).values \
                    -vel['u_kl'].roll(x=-1,y=-1,z=0 ).values + vel['u_kl'].roll(x=0,y=-1,z=0 ).values \
                    -vel['u_kl'].roll(x=-1,y=0 ,z=0 ).values + vel['u_kl'].roll(x=0,y=0 ,z=0 ).values)
    dvdy_kl = facy*(-vel['v_kl'].roll(x=-1,y=-1,z=-1).values - vel['v_kl'].roll(x=0,y=-1,z=-1).values \
                    +vel['v_kl'].roll(x=-1,y=0 ,z=-1).values + vel['v_kl'].roll(x=0,y=0 ,z=-1).values \
                    -vel['v_kl'].roll(x=-1,y=-1,z=0 ).values - vel['v_kl'].roll(x=0,y=-1,z=0 ).values \
                    +vel['v_kl'].roll(x=-1,y=0 ,z=0 ).values + vel['v_kl'].roll(x=0,y=0 ,z=0 ).values)
    dwdz_kl = facz*(-vel['w_kl'].roll(x=-1,y=-1,z=-1).values - vel['w_kl'].roll(x=0,y=-1,z=-1).values \
                    -vel['w_kl'].roll(x=-1,y=0 ,z=-1).values - vel['w_kl'].roll(x=0,y=0 ,z=-1).values \
                    +vel['w_kl'].roll(x=-1,y=-1,z=0 ).values + vel['w_kl'].roll(x=0,y=-1,z=0 ).values \
                    +vel['w_kl'].roll(x=-1,y=0 ,z=0 ).values + vel['w_kl'].roll(x=0,y=0 ,z=0 ).values)
    div_kl = dudx_kl + dvdy_kl + dwdz_kl
    
    # Calculate histograms  
    duhist_kl_np = np.histogram(dudx_kl.flatten(), bins=dbins)
    duhist_kl = scipy.stats.rv_histogram(duhist_kl_np)  
    dvhist_kl_np = np.histogram(dvdy_kl.flatten(), bins=dbins)
    dvhist_kl = scipy.stats.rv_histogram(dvhist_kl_np)  
    dwhist_kl_np = np.histogram(dwdz_kl.flatten(), bins=dbins)
    dwhist_kl = scipy.stats.rv_histogram(dwhist_kl_np) 
    ddivhist_kl_np = np.histogram(div_kl.flatten(), bins=dbins)
    ddivhist_kl = scipy.stats.rv_histogram(ddivhist_kl_np) 
    
    # Store
    all_dudx_hist_kl[i,:] = duhist_kl.pdf(dbins)
    all_dvdy_hist_kl[i,:] = dvhist_kl.pdf(dbins)
    all_dwdz_hist_kl[i,:] = dwhist_kl.pdf(dbins)
    all_div_hist_kl[i,:] = ddivhist_kl.pdf(dbins)
    
# np.save('dudx_ucond_kl_hist.npy', all_dudx_hist_kl.mean(axis=0))
# np.save('dvdy_ucond_kl_hist.npy', all_dvdy_hist_kl.mean(axis=0))
# np.save('dwdz_ucond_kl_hist.npy', all_dwdz_hist_kl.mean(axis=0))
# np.save('div_ucond_kl_hist.npy', all_div_hist_kl.mean(axis=0))